In [95]:
import pandas as pd

In [96]:
churnData = pd.read_csv('/Users/miguelmoreno/Desktop/IRONHACK/week_11_day_2/lab-handling-imbalance/files_for_lab/Customer-Churn.csv')

In [97]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [98]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [99]:
#Usa el argumento errors='coerce' para que pandas convierta esos valores inválidos en NaN 
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors='coerce')

In [100]:
#values faltantes
churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [101]:
#Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges
#Importar todas las librerias necesarias

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [102]:
#Cambiar la columna de churndata por 0 o 1
churnData["Churn"] = churnData["Churn"].replace({"Yes": 1, "No": 0})


/var/folders/tc/dvbccp5x0rzg3_4x3qdj46zh0000gn/T/ipykernel_50797/1703383144.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  churnData["Churn"] = churnData["Churn"].replace({"Yes": 1, "No": 0})


In [103]:
new_churnData = ["tenure", "SeniorCitizen", "MonthlyCharges","TotalCharges"]

In [104]:
#eliminar filas con NANs en columnas
churnData = churnData.dropna(subset=new_churnData + ["Churn"])

In [105]:
#Split the data into a training set and a test set.
X = churnData[new_churnData]
y = churnData["Churn"]

In [106]:
#Scale the features

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [107]:
#Fit a logistic regression model on the training data

# Dividir en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [108]:
X = X.dropna()
y = y[X.index]  # Alinear y con X

In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Entrenar modelo de clasificación
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predecir clases directamente (no probabilidades)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calcular accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Train Accuracy: 0.7934
Test Accuracy: 0.7804


In [110]:
print(churnData["Churn"].value_counts())

Churn
0    5163
1    1869
Name: count, dtype: int64


In [111]:
print(churnData["Churn"].value_counts(normalize=True))

Churn
0    0.734215
1    0.265785
Name: proportion, dtype: float64


In [112]:
# Separar las clases
churn_majority = churnData[churnData["Churn"] == 0]
churn_minority = churnData[churnData["Churn"] == 1]

In [113]:
from sklearn.utils import resample

#Hacer copias aleatorias de los 
#datos de la clase minoritaria para que haya la misma cantidad que en la clase mayoritaria.

churn_minority_upsampled = resample(churn_minority,
                                    replace=True,                  # Muestreo con reemplazo
                                    n_samples=len(churn_majority), # Igualar cantidad con la clase mayoritaria
                                    random_state=42)

# Combinar de nuevo
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])

In [114]:
#Eliminar aleatoriamente datos de la clase mayoritaria para que se iguale con la clase minoritaria.

churn_majority_downsampled = resample(churn_majority,
                                      replace=False,                   # Muestreo sin reemplazo
                                      n_samples=len(churn_minority),  # Igualar cantidad con clase minoritaria
                                      random_state=42)

# Combinar de nuevo
churn_downsampled = pd.concat([churn_majority_downsampled, churn_minority])

In [115]:
print("Upsampled class counts:")
print(churn_upsampled["Churn"].value_counts())

print("\nDownsampled class counts:")
print(churn_downsampled["Churn"].value_counts())

Upsampled class counts:
Churn
0    5163
1    5163
Name: count, dtype: int64

Downsampled class counts:
Churn
0    1869
1    1869
Name: count, dtype: int64


In [117]:
from sklearn.utils import resample

# Concatenar X e y en un mismo DataFrame
churn_balanced = pd.concat([X, y], axis=1)

# Separar clases
churn_majority = churn_balanced[churn_balanced.Churn == 0]
churn_minority = churn_balanced[churn_balanced.Churn == 1]

# Upsample la clase minoritaria
churn_minority_upsampled = resample(
    churn_minority,
    replace=True,
    n_samples=len(churn_majority),
    random_state=42
)

# Combinar datos balanceados
churn_upsampled = pd.concat([churn_majority, churn_minority_upsampled])

In [118]:
X_balanced = churn_upsampled.drop(columns="Churn")
y_balanced = churn_upsampled["Churn"]

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [120]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("Test accuracy:", round(accuracy, 4))

Test accuracy: 0.7256
